In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent.parent

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import modeling
from modeling import ModelingAccessor

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [11]:
gis = GIS(
    url=os.getenv('ESRI_PORTAL_URL'), 
    username=os.getenv('ESRI_PORTAL_USERNAME'),
    password=os.getenv('ESRI_PORTAL_PASSWORD')
)
# gis = GIS(
#     url=os.getenv('ESRI_GIS_URL'), 
#     username=os.getenv('ESRI_GIS_USERNAME'),
#     password=os.getenv('ESRI_GIS_PASSWORD')
# )

cntry = modeling.Country('USA', gis)

cntry

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [12]:
aoi_df = cntry.cbsas.get('Seattle', return_geometry=True)

aoi_df

,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-121.68612849977673, 48.298988999..."


In [13]:
%%time
brnd_df = aoi_df.mdl.business.get_by_name('Ace Hardware')
brnd_df['brand_name_category'] = 'ACE HARDWARE'

brnd_df

CPU times: user 18.9 ms, sys: 1.6 ms, total: 20.5 ms
Wall time: 379 ms


,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,174746248,MAGNOLIA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98199,WA,"{""x"": -122.39802, ""y"": 47.6402850000001, ""spat...",174746248,MAGNOLIA ACE HARDWARE,ACE HARDWARE
1,174841932,ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,LAKE STEVENS,98258,WA,"{""x"": -122.1072885, ""y"": 47.998143, ""spatialRe...",174841932,ACE HARDWARE,ACE HARDWARE
2,174852467,CARRS ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MARYSVILLE,98270,WA,"{""x"": -122.1761745, ""y"": 48.05064, ""spatialRef...",174852467,CARRS ACE HARDWARE,ACE HARDWARE
3,216082099,SOUTH END ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SPANAWAY,98387,WA,"{""x"": -122.4344385, ""y"": 47.087136, ""spatialRe...",216082099,SOUTH END ACE HARDWARE,ACE HARDWARE
4,251373601,CEDAR PLAZA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MOUNTLAKE TER,98043,WA,"{""x"": -122.2900785, ""y"": 47.7909450000001, ""sp...",251373601,CEDAR PLAZA ACE HARDWARE,ACE HARDWARE
5,371889957,GRAHAM ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,GRAHAM,98338,WA,"{""x"": -122.2992495, ""y"": 47.054655, ""spatialRe...",371889957,GRAHAM ACE HARDWARE,ACE HARDWARE
6,401684669,ACE HARDWARE & PAINT CTR,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,TACOMA,98407,WA,"{""x"": -122.5138095, ""y"": 47.27358, ""spatialRef...",401684669,ACE HARDWARE & PAINT CTR,ACE HARDWARE
7,402895226,BURIEN ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,BURIEN,98166,WA,"{""x"": -122.3411805, ""y"": 47.4694920000001, ""sp...",402895226,BURIEN ACE HARDWARE,ACE HARDWARE
8,403513308,ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,EVERETT,98203,WA,"{""x"": -122.2132808, ""y"": 47.9534799, ""spatialR...",403513308,ACE HARDWARE,ACE HARDWARE
9,403876556,STANWOOD ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,STANWOOD,98292,WA,"{""x"": -122.3293095, ""y"": 48.236229, ""spatialRe...",403876556,STANWOOD ACE HARDWARE,ACE HARDWARE


In [14]:
%%time
comp_df = aoi_df.mdl.business.get_competition(brnd_df, local_threshold=3)

comp_df

CPU times: user 27.2 ms, sys: 2.53 ms, total: 29.7 ms
Wall time: 439 ms


,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MC LENDON HARDWARE,local_brand
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,DUVALL TRUE VALUE HARDWARE,local_brand
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,GATEWAY TRUE VALUE HARDWARE,local_brand
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC


In [15]:
brand_repl_lst = [
    (r'TRUE VALUE|TRUE VL', 'TRUE VALUE'),
    (R'MC LENDON|MCLENDON' , 'MCLENDON HARDWARE')
]

for search_re, repl_str in brand_repl_lst:
    
    brand_filter = comp_df.brand_name.str.contains(search_re, regex=True)
    
    comp_df.loc[brand_filter, 'brand_name'] = repl_str
    
comp_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MCLENDON HARDWARE,local_brand
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE,MCLENDON HARDWARE INC
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,TRUE VALUE,local_brand
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,TRUE VALUE,local_brand
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE,MCLENDON HARDWARE INC


In [16]:
comp_df.mdl.business.calculate_brand_name_category(3, inplace=True)

comp_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MCLENDON HARDWARE,MCLENDON HARDWARE
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE,MCLENDON HARDWARE
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,TRUE VALUE,TRUE VALUE
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,TRUE VALUE,TRUE VALUE
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE,MCLENDON HARDWARE


In [17]:
brnd_df['brand_comp'] = 'brand'
comp_df['brand_comp'] = 'competition'

In [18]:
loc_df = pd.concat([brnd_df, comp_df])
loc_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category,brand_comp
0,174746248,MAGNOLIA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98199,WA,"{""x"": -122.39802, ""y"": 47.6402850000001, ""spat...",174746248,MAGNOLIA ACE HARDWARE,ACE HARDWARE,brand
1,174841932,ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,LAKE STEVENS,98258,WA,"{""x"": -122.1072885, ""y"": 47.998143, ""spatialRe...",174841932,ACE HARDWARE,ACE HARDWARE,brand
2,174852467,CARRS ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MARYSVILLE,98270,WA,"{""x"": -122.1761745, ""y"": 48.05064, ""spatialRef...",174852467,CARRS ACE HARDWARE,ACE HARDWARE,brand
3,216082099,SOUTH END ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SPANAWAY,98387,WA,"{""x"": -122.4344385, ""y"": 47.087136, ""spatialRe...",216082099,SOUTH END ACE HARDWARE,ACE HARDWARE,brand
4,251373601,CEDAR PLAZA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MOUNTLAKE TER,98043,WA,"{""x"": -122.2900785, ""y"": 47.7909450000001, ""sp...",251373601,CEDAR PLAZA ACE HARDWARE,ACE HARDWARE,brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand,competition
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand,competition
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand,competition
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE,MCLENDON HARDWARE,competition


In [19]:
loc_df.brand_name_category.value_counts().sort_values(ascending=False)

local_brand          58
ACE HARDWARE         37
TRUE VALUE           10
MCLENDON HARDWARE     7
Name: brand_name_category, dtype: int64